In [1]:
import numpy as np
import pyreadr
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], "..")) # to import from parent directory
from utils.utils import generate_support_points
from evaluation.metrics import extremal_coefficient

# Save Single model

By specifying the model and path, one can transform simulations from the R.script into corresponding numpy files, useable for pytorch.

In [3]:
dir = "application"
path = f'../data/{dir}/data/'
# Image size
img_size = 30
models = ["brown", "powexp", "whitmat"]

## Transform train data

In [6]:
# Train/val split
split = 4000

for model in models:
    params = pyreadr.read_r(path+f"{model}_train_params.RData")["train_params"]
    data = pyreadr.read_r(path+f"{model}_train_data.RData")["train_data"].to_numpy()
    #Train/val
    train_params = params.loc[0:(split-1),].to_numpy()
    val_params =params.loc[split:,].to_numpy()
    train_data =data[:,0:split]
    train_data = np.reshape(np.swapaxes(train_data, 0,1), newshape = (-1, img_size, img_size))
    val_data = data[:,split:]
    val_data = np.reshape(np.swapaxes(val_data, 0,1), newshape = (-1, img_size, img_size))
    
    np.save(path+f"{model}_train_params", train_params)
    np.save(path+f"{model}_val_params", val_params)
    np.save(path+f"{model}_train_data", train_data)
    np.save(path+f"{model}_val_data", val_data)

## Transform test data

In [ ]:
for model in models:
    test_params = pyreadr.read_r(path+f"{model}_test_params.RData")["test_params"].to_numpy()
    test_data = pyreadr.read_r(path+f"{model}_test_data.RData")["test_data"].to_numpy()
    test_data = np.reshape(np.swapaxes(test_data, 0,1), newshape = (-1, img_size, img_size))

    np.save(path+f"{model}_test_params", test_params)
    np.save(path+f"{model}_test_data", test_data)

# Save aggregated model

Put several models together for training data and save them as one. For this case, one also needs to extract the extremeal coefficient function beforehand. The test data stays the same.

In [2]:
dir = "application"
path = f'../data/{dir}/data/'
# Image size
img_size = 30
models = ["brown", "powexp", "whitmat"]

# Parameters for extremal coefficient function
h_support = generate_support_points(dh = 0.1, max_length = 42.5)

In [24]:
# Train/val split (individual per model)
test_size = 5000
split = 4000
n_models = len(models)

train_params = np.zeros((split * n_models, 2))
val_params = np.zeros(((test_size - split) * n_models, 2))
train_data = np.zeros((split * n_models, img_size, img_size))
val_data = np.zeros(((test_size - split) * n_models, img_size, img_size))
train_ext_coef = np.zeros((split * n_models, len(h_support)))
val_ext_coef = np.zeros(((test_size - split) * n_models, len(h_support)))


for i, model in enumerate(models):
    # Generate data and parameters
    params = pyreadr.read_r(path+f"{model}_train_params.RData")["train_params"]
    data = pyreadr.read_r(path+f"{model}_train_data.RData")["train_data"].to_numpy()
    #Train/val
    train_params[i*split:(i+1)*split] = params.to_numpy()[0:split]
    val_params[i*(test_size -split):(i+1)*(test_size-split)] = params.to_numpy()[split:,]
    train_data[i*split:(i+1)*split,:,:] = np.reshape(np.swapaxes(data[:,0:split], 0,1), newshape = (-1, img_size, img_size))
    val_data[i*(test_size -split):(i+1)*(test_size-split),:,:] = np.reshape(np.swapaxes(data[:,split:], 0,1), newshape = (-1, img_size, img_size))

    # Calculate extremal coefficient function
    train_ext_coef[i*split:(i+1)*split] = extremal_coefficient(h = h_support, model = model, r = params.to_numpy()[0:split,0], s = params.to_numpy()[0:split,1])
    val_ext_coef[i*(test_size -split):(i+1)*(test_size-split)] = extremal_coefficient(h = h_support, model = model, r = params.to_numpy()[split:,0], s = params.to_numpy()[split:,1])
    
np.save(path+f"aggregated_train_params", train_params)
np.save(path+f"aggregated_val_params", val_params)
np.save(path+f"aggregated_train_data", train_data)
np.save(path+f"aggregated_val_data", val_data)
np.save(path+f"aggregated_train_ext_coef", train_ext_coef)
np.save(path+f"aggregated_val_ext_coef", val_ext_coef)